In [12]:
!pip install datasets tensorflow keras

In [13]:
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import collections

In [14]:
# -------------------------------------------------------------
# 1. LOAD TWEETEVAL SENTIMENT DATASET (3 CLASSES)
#    Labels: 0 = Negative, 1 = Neutral, 2 = Positive
# -------------------------------------------------------------
dataset = load_dataset("tweet_eval", "sentiment")

train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]

val_texts   = dataset["validation"]["text"]
val_labels  = dataset["validation"]["label"]

test_texts  = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

print("✅ Dataset loaded!")
print("Train samples:", len(train_texts))
print("Val samples:  ", len(val_texts))
print("Test samples: ", len(test_texts))

✅ Dataset loaded!
Train samples: 45615
Val samples:   2000
Test samples:  12284


In [15]:
# -------------------------------------------------------------
# 1A. INSPECT SOME DATA
# -------------------------------------------------------------
print("\n🔍 SAMPLE TRAINING TWEETS (first 10):")
for i in range(10):
    print(f"Text : {train_texts[i]}")
    print(f"Label: {train_labels[i]}")  # 0=neg, 1=neutral, 2=pos
    print("-" * 60)


🔍 SAMPLE TRAINING TWEETS (first 10):
Text : "QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"
Label: 2
------------------------------------------------------------
Text : "Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ"
Label: 1
------------------------------------------------------------
Text : Sorry bout the stream last night I crashed out but will be on tonight for sure. Then back to Minecraft in pc tomorrow night.
Label: 1
------------------------------------------------------------
Text : Chase Headley's RBI double in the 8th inning off David Price snapped a Yankees streak of 33 consecutive scoreless innings against Blue Jays
Label: 1
------------------------------------------------------------
Text : @user Alciato: Bee will invest 150 million in January, another 200 in the Summer and plans to bring Messi by 2017"
Label: 2
-----------------------------------------------------

In [16]:
# Label distribution
label_counts = collections.Counter(train_labels)
print("\n📊 LABEL DISTRIBUTION (TRAIN SET):")
print("0 = Negative, 1 = Neutral, 2 = Positive\n")
for label, count in label_counts.items():
    print(f"Label {label}: {count} samples")


📊 LABEL DISTRIBUTION (TRAIN SET):
0 = Negative, 1 = Neutral, 2 = Positive

Label 2: 17849 samples
Label 1: 20673 samples
Label 0: 7093 samples


In [17]:
# -------------------------------------------------------------
# 2. TOKENIZE TEXT + PAD SEQUENCES
# -------------------------------------------------------------
vocab_size = 20000   # cap vocabulary size for speed & stability
max_length = 80      # max tokens per tweet (after padding/truncation)

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# Text → integer sequences
X_train = tokenizer.texts_to_sequences(train_texts)
X_val   = tokenizer.texts_to_sequences(val_texts)
X_test  = tokenizer.texts_to_sequences(test_texts)

# Pad to fixed length for LSTM input
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val   = pad_sequences(X_val,   maxlen=max_length, padding='post', truncating='post')
X_test  = pad_sequences(X_test,  maxlen=max_length, padding='post', truncating='post')

# One-hot encode labels (3 classes)
y_train = to_categorical(train_labels, num_classes=3)
y_val   = to_categorical(val_labels,   num_classes=3)
y_test  = to_categorical(test_labels,  num_classes=3)

print("\n✅ Tokenization & padding complete.")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


✅ Tokenization & padding complete.
X_train shape: (45615, 80)
y_train shape: (45615, 3)


In [18]:
# -------------------------------------------------------------
# 3. BUILD Bidirectional LSTM MODEL WITH DROPOUT
# -------------------------------------------------------------
embedding_dim = 128

model = Sequential([
    Input(shape=(max_length,)),                     # (batch, 40)
    Embedding(vocab_size, embedding_dim),           # → (batch, 40, 128)

    # Bidirectional LSTM: 64 units per direction → 128-dim combined
    Bidirectional(
        LSTM(
            64,
            dropout=0.3,             # dropout on inputs
            recurrent_dropout=0.0    # keep 0.0 for speed; 0.3 is slower but possible
        )
    ),

    Dense(64, activation='relu'),
    Dropout(0.3),                                   # extra dropout on dense layer
    Dense(3, activation='softmax')                  # 3-class sentiment
])

In [19]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

print("\n🧠 Model Summary:")
model.summary()


🧠 Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 80, 128)        │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,667,267 (10.17 MB)

 Trainable params: 2,667,267 (10.17 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# -------------------------------------------------------------
# 4. EARLY STOPPING (ANTI-OVERFITTING)
# -------------------------------------------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

In [21]:
# -------------------------------------------------------------
# 5. TRAIN THE MODEL (BiLSTM)
# -------------------------------------------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,              # early stopping will usually stop around 3–5
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/10
713/713 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.5312 - loss: 0.9173 - val_accuracy: 0.6690 - val_loss: 0.7478
Epoch 2/10
713/713 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.7195 - loss: 0.6443 - val_accuracy: 0.6800 - val_loss: 0.7294
Epoch 3/10
713/713 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.7803 - loss: 0.5195 - val_accuracy: 0.6535 - val_loss: 0.7891
Epoch 4/10
713/713 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8193 - loss: 0.4310 - val_accuracy: 0.6510 - val_loss: 0.9202


In [22]:
# -------------------------------------------------------------
# 6. EVALUATE ON TEST SET
# -------------------------------------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f"\n🔥 BiLSTM Test Accuracy: {acc * 100:.2f}%")

384/384 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6147 - loss: 0.8663

🔥 BiLSTM Test Accuracy: 61.36%
